#### ID 9702

```Count the number of unique facilities per municipality zip code along with the number of inspections. Output the result along with the number of inspections per each municipality zip code. Sort the result based on the number of inspections in descending order.```

In [ ]:
%%sql
SELECT facility_zip,
       COUNT(DISTINCT facility_id) AS no_facilities,
       COUNT(*)                    AS no_inspections
FROM los_angeles_restaurant_health_inspections
GROUP BY facility_zip
ORDER BY no_inspections DESC

In [ ]:
df = los_angeles_restaurant_health_inspections

df.groupby('facility_zip').agg(no_facilities=('facility_id', 'nunique'),
                               no_inspections=('facility_id', 'count')).sort_values('no_inspections', ascending=False)

#### ID 9703

```Find the most common grade earned by bakeries.```

In [ ]:
%%sql
WITH cte AS (SELECT grade,
                    COUNT(*)                                      n_bakeries,
                    DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS rnk
             FROM los_angeles_restaurant_health_inspections
             WHERE facility_name ILIKE '%BAKERY%'
             GROUP BY grade)
SELECT grade
FROM cte
WHERE rnk = 1

In [ ]:
df = los_angeles_restaurant_health_inspections

df.query('facility_name.str.contains("bakery", regex=True, case=False)').groupby('grade', as_index=False).agg(
    n_bakeries=('serial_number', 'count')).nlargest(1, 'n_bakeries', keep='all')['grade']

### ID 9704

```Find the number of inspections per day. Output the result along with the date of the activity. Order results based on the activity date in the ascending order.```

In [ ]:
%%sql
SELECT activity_date, COUNT(*) AS n_inspections
FROM los_angeles_restaurant_health_inspections
GROUP BY activity_date
ORDER BY activity_date

In [ ]:
f = los_angeles_restaurant_health_inspections

df['new_activity_date'] = df['activity_date'].dt.date

df.groupby('new_activity_date', as_index=False).agg(n_inspections=('activity_date', 'count')).sort_values(
    'new_activity_date')

#### ID 9707

```Find the mean of inspections scores between 91 and 100. Assuming that the scores are normally distributed.```

In [ ]:
%%sql
SELECT AVG(score) AS avg_score
FROM los_angeles_restaurant_health_inspections
WHERE score BETWEEN 91 AND 100

In [ ]:
df = los_angeles_restaurant_health_inspections

df.query('score.between(91, 100)')['score'].mean()

#### ID 9717

```Find all businesses which have low-risk safety violations.```

In [ ]:
%%sql
SELECT DISTINCT business_name
FROM sf_restaurant_health_violations
WHERE risk_category = 'Low Risk'

In [ ]:
df = sf_restaurant_health_violations

df.query('risk_category == "Low Risk"')['business_name'].drop_duplicates()

#### ID 9718

```Find all businesses which have a phone number.```

In [ ]:
%%sql
SELECT DISTINCT business_name
FROM sf_restaurant_health_violations
WHERE business_phone_number IS NOT NULL

In [ ]:
df = sf_restaurant_health_violations

df.query('~business_phone_number.isnull()')['business_name'].drop_duplicates()

#### ID 9719

```Find all business postal codes of restaurants with issues related to the water (violation description contains substring "water").```

In [ ]:
%%sql
SELECT DISTINCT business_postal_code
FROM sf_restaurant_health_violations
WHERE violation_description ILIKE '%water%'

In [ ]:
df = sf_restaurant_health_violations

df.query('~violation_description.isnull() & violation_description.str.contains("water")')[
    'business_postal_code'].drop_duplicates()